In [1]:
import numpy as np
import scipy

np.set_printoptions(precision=4, suppress=True, linewidth=260)

In [2]:
import veloxchem as vlx

In [3]:
h2o_xyz = """3

O    0.000000000000        0.000000000000        0.000000000000
H    0.000000000000        0.740848095288        0.582094932012
H    0.000000000000       -0.740848095288        0.582094932012
"""

molecule = vlx.Molecule.read_xyz_string(h2o_xyz)
basis = vlx.MolecularBasis.read(molecule, "cc-pVDZ", ostream=None)

In [4]:
norb = basis.get_dimensions_of_basis()
nocc = molecule.number_of_alpha_electrons()

print("Number of contracted basis functions:", norb)
print("Number of doubly occupied molecular orbitals:", nocc)

Number of contracted basis functions: 24
Number of doubly occupied molecular orbitals: 5


In [5]:
scf_drv = vlx.ScfRestrictedDriver()
scf_drv.ostream.mute()

scf_results = scf_drv.compute(molecule, basis)

In [6]:
F = scf_results["F_alpha"]
S = scf_results["S"]

In [7]:
print(f"Orbital energies of occupied orbitals: {scf_results['E_alpha'][:nocc]}")

Orbital energies of occupied orbitals: [-20.5482  -1.3452  -0.7058  -0.5711  -0.4946]


In [8]:
epsilon, C = scipy.linalg.eigh(F, S)

In [9]:
print(f"Orbital energies of occupied orbitals: {epsilon[:nocc]}")

Orbital energies of occupied orbitals: [-20.5482  -1.3452  -0.7058  -0.5711  -0.4946]


In [10]:
S_ext = np.zeros((norb + 1, norb + 1))

S_ext[1 : norb + 1, 1 : norb + 1] = S

S_ext[0, 1 : norb + 1] = S[0, :]
S_ext[1 : norb + 1, 0] = S[:, 0]

S_ext[0, 0] = S[0, 0]

In [11]:
F_ext = np.zeros((norb + 1, norb + 1))

F_ext[1 : norb + 1, 1 : norb + 1] = F

F_ext[0, 1 : norb + 1] = F[0, :]
F_ext[1 : norb + 1, 0] = F[:, 0]

F_ext[0, 0] = F[0, 0]

In [12]:
try:
    epsilon, C = scipy.linalg.eigh(F_ext, S_ext)
except:
    print("Error: Linear dependence in the atomic orbital basis!")

Error: Linear dependence in the atomic orbital basis!


In [13]:
sigma_ext, U_ext = np.linalg.eigh(S_ext)

In [14]:
np.isclose(0.0, sigma_ext)

array([ True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False])

In [15]:
U = U_ext[:, 1:]
sigma = sigma_ext[1:]

In [16]:
U.shape

(25, 24)

In [17]:
# canonical transformation
X = np.einsum("ik,k->ik", U, 1 / np.sqrt(sigma))

In [18]:
F_OAO = np.einsum("ki,kl,lj->ij", X, F_ext, X)
epsilon, C_OAO = np.linalg.eigh(F_OAO)

In [19]:
C = np.einsum("ik,kj->ij", X, C_OAO)

In [20]:
print(f"Orbital energies of occupied orbitals: {epsilon[:nocc]}")

Orbital energies of occupied orbitals: [-20.5482  -1.3452  -0.7058  -0.5711  -0.4946]
